In [1]:
import numpy as np
from qutip import*
import time
import matplotlib.pyplot as plt
import itertools as it
sqrt = np.sqrt
pi   = np.pi
class Add_gate:
    def __init__(gate, name, Tar_Con, angle):
        gate.name    = name
        gate.Tar_Con = Tar_Con
        gate.angle   = angle
                
Nqubits= 3
Nlevels= 3
Paulis_gt = 4.00e-8
CZ_gt = 2.83e-7
gate_time_CCZS = CZ_gt/(sqrt(2))
Diss = [27.35e-6, 39.73e-6, 34.73e-6]
Alpha = -2*pi*210e6
ZZ_list = [[0,1]]
ZZ_strength = [0*2*pi*100e3]

In [2]:
def project_to_qubit(rho):
    N = int(len(rho.dims[0]))
    projector = Qobj(np.array([[1, 0, 0],[0, 1, 0]]))
    tot_proj = tensor([projector]*N)
    rho_proj = tot_proj*rho*tot_proj.dag()
    return rho_proj.unit()

In [3]:
T1s = [27.35e-6, 39.73e-6, 34.73e-6]
T2s = [45.21e-6, 57.67e-6, 21.49e-6]

Deph = []
for i in range(3):
    _1byT2 = 1/T2s[i]
    _1byT1 = 1/(2*T1s[i])
    Deph.append(1/(_1byT2 - _1byT1))

In [4]:
# Import the developed package
import Chalmers_Simulator_mcsolve as CS

The quantum gates which are modelled in this code and their notations-
--------------------------------------------------------------------------------
Pauli X 		 PX
Pauli Y 		 PY
Pauli Z 		 PZ
Hadamard 		 HD
PI12 			 1->2 transition


Controlled Z 		 CZ 		 Format:Tar_Con=[[control, target]]
Controlled CZS 		 CCZS 		 Format:Tar_Con=[[control, target1, target2, phi]]
Sqrt Controlled CZS 	 SCCZS 		 Format:Tar_Con=[[control, target1, target2, phi]]


In [5]:
# Create Hamiltonian and collapse operators
Hamiltonian, c_ops = CS.create_system_Hamiltonian(Nqubits, Nlevels, Paulis_gt, CZ_gt, gate_time_CCZS, Alpha, Diss, \
                                                  Deph, Texc = [], ZZ_list=[], ZZ_strength=[])

In [16]:
gate=[]
gate.append(Add_gate(name=["HD" , "HD"], Tar_Con=[0,1],   angle=[pi/2, pi/2]           ))
Ini = tensor(basis(3,0), basis(3,0), basis(3,0))

In [17]:
def main_part():
    fstate = CS.Execute(Hamiltonian, c_ops, gate, Ini)
    return fstate

In [ ]:
ntraj = 100
states = parallel_map(main_part, range(ntraj), num_cpus=7, progress_bar=True)

In [20]:
def Evalues(e_ops, states):
    '''
    This function calculates the average values of operators from all the
    final states calculated from the trajectory approach.
    
    Arguments
    --------------------
    e_ops         :       List of operators
    states        :       A list of final states
    
    
     Returns
    --------------------
    Epect_values  :       A list of expectation values corresponding to the
                          trajectories for each operators in e_ops
    
    
    '''
    
    
    Epect_values = []
    for i in e_ops:
        expt = []
        for j in range(len(states)):
            expt.append(expect(i, states[j]))
        Epect_values.append(np.mean(expt))
    return Epect_values